## 1. Descripción del corpus

### Librerias

In [49]:
# Librerias
import re
import nltk
import ftfy
import pandas as pd

from collections import Counter
from nltk.corpus import stopwords

In [51]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/diego23/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Número de documentos, tokens y vocabulario.

In [15]:
# Configuracion
csv_path = "../data/raw/MeIA_2025_train.csv"
text_col = "Review"
classes = ["Polarity", "Town", "Region", "Type"]

# Cargamos el dataset y elegimos codificacion
df = pd.read_csv(csv_path, encoding='utf-8')

# Limpiamos un poco
df = df.dropna(subset=[text_col])                 # Elimina valores NaN
df = df[df[text_col].str.strip().ne("")]          # Elimina strings vacios
df.reset_index(drop=True, inplace=True)

# Imprimimos resultados
print("Columnas disponibles:", df.columns.tolist())
print("Número de documentos:", len(df))

# Visualizamos el df
df.head()

Columnas disponibles: ['Review', 'Polarity', 'Town', 'Region', 'Type']
Número de documentos: 5000


,Review,Polarity,Town,Region,Type
0,Un Restaurante te invita por su ambiente tan a...,2.0,Tlaquepaque,Jalisco,Restaurant
1,Pagamos 25 pesos por la entrada y no es gran c...,3.0,Bacalar,QuintanaRoo,Attractive
2,Mi esposa y yo nos alojamos en el Dreams por 4...,3.0,Tulum,QuintanaRoo,Hotel
3,"La única decepción puede no ser José Cuervo, p...",2.0,Tequila,Jalisco,Attractive
4,Cuando leí los comentarios sobre cómo son las ...,1.0,Isla_Mujeres,QuintanaRoo,Hotel


In [ ]:
# Contamos los mojibakes "Ã"
count_before = df[text_col].astype(str).str.count("Ã").sum()
print(f"Número de 'Ã' antes de ftfy: {count_before}")

# Aplicamos ftfy
df["Review_clean"] = df[text_col].apply(ftfy.fix_text)

# Contamos de nuevo después
count_after = df["Review_clean"].astype(str).str.count("Ã").sum()
print(f"Número de 'Ã' después de ftfy: {count_after}")

# Removemos casos tipo "...Mas" al final del texto por scrapeo
def remove_scrape_artifacts(text: str):
    if not isinstance(text, str):
        return text
    return re.sub(r"Más[\s\W]*$", "", text)
# Aplicamos
df["Review_clean"] = df["Review_clean"].apply(remove_scrape_artifacts)

# Verificamos
examples_after = df[df["Review_clean"].str.contains(r"\.\.\..*Mas", na=False)]
print("Ejemplos de artefactos restantes después de limpiar:", len(examples_after))

Número de 'Ã' antes de ftfy: 731
Número de 'Ã' después de ftfy: 0
Ejemplos de artefactos restantes después de limpiar: 3


In [44]:
# Creamos una expresion regular para tokenizar palabras en español
WORD_RE = re.compile(r"[A-Za-zÁÉÍÓÚÜÑáéíóúüñ]+(?:[-'][A-Za-zÁÉÍÓÚÜÑáéíóúüñ]+)?")

# Realizamos la tokenizacion
def tokenize(text: str):
    if not isinstance(text, str):
        return []
    text = text.lower()  # Pasamos a minúsculas
    return WORD_RE.findall(text)

# Aplicamos tokenización sobre la columna limpia
df["tokens"] = df["Review_clean"].apply(tokenize)
df["n_tokens"] = df["tokens"].apply(len)

# Contamos tokens en todo el corpus
token_counts = Counter()
for toks in df["tokens"]:
    token_counts.update(toks)

# Total de tokens (con repeticiones)
total_tokens = sum(token_counts.values())

# Vocabulario (palabras únicas)
vocab_size = len(token_counts)

print(f"Total de tokens en el corpus: {total_tokens}")
print(f"Tamaño del vocabulario: {vocab_size}")

Total de tokens en el corpus: 348935
Tamaño del vocabulario: 19517


### Hapax legomena y su proporción.

In [48]:
# Hapax = términos con frecuencia 1
hapax_list = [t for t, c in token_counts.items() if c == 1]
hapax_count = len(hapax_list)

# Proporción de hapax sobre vocabulario (qué fracción del vocab son hapax)
hapax_prop_over_vocab = hapax_count / max(len(token_counts), 1)

# Proporción de hapax sobre tokens (qué fracción de todos los tokens son hapax)
hapax_prop_over_tokens = hapax_count / max(total_tokens, 1)

print(f"Hapax (freq=1): {hapax_count}")
print(f"Proporción hapax / vocabulario: {hapax_prop_over_vocab:.4f}")
print(f"Proporción hapax / tokens: {hapax_prop_over_tokens:.4f}")

# (Opcional) Ver algunos ejemplos de hapax
pd.Series(hapax_list[:20], name="Ejemplos de hapax (20)").to_frame()

Hapax (freq=1): 9957
Proporción hapax / vocabulario: 0.5102
Proporción hapax / tokens: 0.0285


,Ejemplos de hapax (20)
0,apuran
1,descuidaron
2,pesas
3,frustrado
4,acanalado
5,elástico
6,ceñir
7,púrpura
8,cintura
9,blye


### Porcentaje de stopwords.

In [53]:
# Lista de stopwords en español
stopwords_es = set(stopwords.words("spanish"))

# Función para detectar stopwords
def is_stopword(tok: str):
    return tok in stopwords_es

# Contamos stopwords en cada documento
df["stop_tokens"] = df["tokens"].apply(lambda toks: [t for t in toks if is_stopword(t)])
df["n_stop"] = df["stop_tokens"].apply(len)

# Totales
total_stop = df["n_stop"].sum()
stopword_pct = (total_stop / total_tokens) * 100

print(f"Total de stopwords en el corpus: {total_stop}")
print(f"Porcentaje de stopwords sobre tokens: {stopword_pct:.2f}%")


Total de stopwords en el corpus: 175571
Porcentaje de stopwords sobre tokens: 50.32%


### Estadı́sticas por clase (número de documentos, tokens y vocabulario).

In [ ]:
present_cols = [c for c in classes if c in df.columns]

def per_group_stats(group_df: pd.DataFrame) -> pd.Series:
    # n_docs
    n_docs = len(group_df)
    # n_tokens y vocab del grupo
    counter = Counter()
    for toks in group_df["tokens"]:
        counter.update(toks)
    n_tokens = sum(counter.values())
    vocab_size = len(counter)
    return pd.Series({"n_docs": n_docs, "n_tokens": n_tokens, "vocab_size": vocab_size})

# Aplicar a cada columna de clase presente
stats_list = []
for col in present_cols:
    stats = (
        df.groupby(col, dropna=False)
          .apply(per_group_stats, include_groups=False)
          .reset_index()
          .rename(columns={col: "class_value"})
          .assign(class_col=col)  # para saber de qué columna viene
          .loc[:, ["class_col", "class_value", "n_docs", "n_tokens", "vocab_size"]]
    )
    stats_list.append(stats)

stats_per_class = pd.concat(stats_list, ignore_index=True)

# Mostrar resultados
stats_per_class.head(20)


,class_col,class_value,n_docs,n_tokens,vocab_size
0,Polarity,1.0,800,61908,7670
1,Polarity,2.0,900,74973,8421
2,Polarity,3.0,1000,69431,7594
3,Polarity,4.0,1100,68641,7319
4,Polarity,5.0,1200,73982,8304
5,Town,Ajijic,102,7361,1900
6,Town,Atlixco,48,3355,1172
7,Town,Bacalar,250,13449,2685
8,Town,Bernal,58,3055,1013
9,Town,Chiapa_de_Corzo,30,2243,818


In [56]:
# Guardamos estadísticas por clase en processed
output_path = "../data/processed/stats_per_class.csv"
stats_per_class.to_csv(output_path, index=False)
print(f"Archivo guardado en: {output_path}")

Archivo guardado en: ../data/processed/stats_per_class.csv


## 2. Ley de Zipf 

### Calcula la frecuencia absoluta $f(w)$ de cada palabra $w$ en el corpus y ordénalas de mayor a menor. A cada palabra asi ordenada se le asigna un rango $r$, donde $r = 1$ corresponde a la palabra más frecuente, $r = 2$ a la segunda, y asi sucesivamente.

## 3. Palabras importantes por clase

## 4. Patrones gramaticales (POS 4-gramas)

## 5. Representaciones BoW

## 6. Bigramas

## 7. Word2Vec y analogı́as

## 8. Embeddings de documento y clusterización

## 9. Clasificación con partición 70/30

## 10. LSA con 50 tópicos